

```
# This is formatted as code
```

# Demo DeepFake

**Clone repository**

In [1]:
!rm -rf first-order-model
!git clone https://github.com/AliaksandrSiarohin/first-order-model


Cloning into 'first-order-model'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 243 (delta 0), reused 2 (delta 0), pack-reused 239
Receiving objects: 100% (243/243), 71.46 MiB | 41.74 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [2]:
pwd

'/content'

In [3]:
cd first-order-model

/content/first-order-model


In [4]:
ls 

animate.py       demo.ipynb         logger.py          run.py
augmentation.py  demo.py            modules/           sup-mat/
config/          Dockerfile         README.md          sync_batchnorm/
crop-video.py    frames_dataset.py  reconstruction.py  train.py
data/            LICENSE.md         requirements.txt


**Mount your Google drive folder on Colab**

In [5]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.**

**Load driving video and source image**

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")
path = '/content/gdrive/My Drive/DeepFakeInterviewMe/first-order-motion-model/'
source_image = imageio.imread(path + 'julio5.png')

reader = imageio.get_reader(path +  'fox1.mp4','ffmpeg')
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [7]:
import sys
sys.path.append('/content/first-order-motion-model/')
import demo
#print(demo.hola)
generator, kp_detector = demo.load_checkpoints(config_path='config/vox-256.yaml',checkpoint_path= path + 'vox-cpk.pth.tar')

**Perform image animation**

In [8]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 659/659 [00:24<00:00, 26.65it/s]
